### Code sandbox for elastic wave equation

**Hello there**

### Let's take a look at Devito's Elastic Example

The elastic example can be found [here](https://github.com/devitocodes/devito/tree/master/examples/seismic/elastic). Some equations I found on there - 

$$
\frac{\partial}{\partial t} v - \frac{1}{\rho} \nabla \mathbf{.} \tau = 0
$$

**Note:** RHS should be a source term(?) maybe not 0
$$
\frac{\partial}{\partial t} \tau - \lambda \text{diag}(\nabla . v) - \mu \left( \nabla v + (\nabla v)^T \right) = 0
$$

Let's compare this with the derivation [here](https://wiki.seg.org/wiki/Mathematical_foundation_of_elastic_wave_propagation#math_L-34).

Different formulation here, assumes $\lambda$ and $\mu$ are fixed - 
$$
\rho \frac{\partial^2 \mathbf{u}}{\partial t^2} = (\lambda + \mu) \nabla \Delta + \mu \nabla^2 \mathbf{u} \;\;\;\; \mathbf{(1)}
$$

$$
\mathbf{u} = 
\begin{pmatrix}
    u\\
    v\\
    w\\
\end{pmatrix},\;\;\; \Delta = \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} + \frac{\partial w}{\partial z} \right)
$$

with some effort, we can convert to this format - 

$$
\rho \frac{\partial^2 \Delta}{\partial t^2} = (\lambda + \mu) \nabla^2 \Delta  \;\;\;\; \mathbf{(2)}
$$

this is called the *dilatational* or *compressional-wave* propagation. It can be derived from the earlier succinctly by noting that 
$$
\Delta = \nabla  \mathbf{. u}
$$

This represents a compressional wave propagating with a speed 

$$
v_{p} = \alpha = \sqrt{\frac{(\lambda + \mu)}{\rho}}
$$

We can also define a rotational vector 
$$
\Theta := \begin{pmatrix}
\theta_{xy}\\
\theta_{xz}\\
\theta_{yz}
\end{pmatrix}
$$
and take the curl of **(1)**, to get

$$
\rho \frac{\partial^2}{\partial t^2} \Theta = \mu \nabla^{2} \Theta \;\;\;\;\; \mathbf{(3)}
$$
propagating at speed
$$
v_{s} = \beta = \sqrt{\frac{\mu}{\rho}}
$$

**(2)** represents the P-wave, and **(3)** represents the S-wave propagation. We should now have enough information to figure out what that Devito example is doing.

To recap, here's what Devito does - 

$$
\frac{\partial}{\partial t} \mathbf{v} - \frac{1}{\rho} \nabla \mathbf{.} \tau = 0 \;\;\;\; \mathbf{(4)}
$$

$$
\frac{\partial}{\partial t} \tau - \lambda \text{diag}(\nabla . \mathbf{v}) - \mu \left( \nabla \mathbf{v} + (\nabla \mathbf{v})^T \right) = 0
\;\;\;\; \mathbf{(5)}
$$
**Note**: diag(x) means form a diagonal matrix, using the entries of x on the principal diagonal

To get **(4)** from **(1)**, first we define 
$$
\mathbf{v} = \frac{\partial}{\partial t} \mathbf{u} \;\;\;\;\;\; \tau = ?
$$

Now replace $\mathbf{u}$ with $v$, and divide by $\rho$ in the equation to get
$$
\frac{\partial \mathbf{v}}{\partial t} = \frac{1}{\rho}\left( (\lambda + \mu) \nabla \Delta + \mu \nabla^2 \mathbf{u} \right)
$$
therefore
$$
\nabla \mathbf{.} \tau = \left( (\lambda + \mu) \nabla \Delta + \mu \nabla^2 \mathbf{u} \right)
$$

for more insight on what $\tau$ even is, we can look at **(5)** and write it in terms of $\mathbf{u}$ - 
$$
\frac{\partial}{\partial t} \tau - \lambda \text{diag}(\nabla . \frac{\partial \mathbf{u}}{\partial t}) - \mu \left( \nabla \frac{\partial \mathbf{u}}{\partial t} + \left(\nabla \frac{\partial \mathbf{u}}{\partial t}\right)^T \right) = 0
$$

$$
\implies \tau - \lambda \text{diag}\left(\nabla \mathbf{. u} \right) - \mu (\nabla \mathbf{u} + (\nabla \mathbf{u})^T) = 0
$$

$$
\implies \tau = \lambda \text{diag}\left(\nabla \mathbf{. u} \right) + \mu (\nabla \mathbf{u} + (\nabla \mathbf{u})^T)
$$

$$
\implies \tau = \lambda \text{diag}\left(\nabla \mathbf{. u} \right) + 2\mu (\mathbf{e_{ij}})
$$

$$
\implies \tau = \lambda . 
\begin{pmatrix}
{\frac{\partial u}{\partial x}} & {0} & {0} \\
{0} & {\frac{\partial v}{\partial y}} & {0} \\
{0} & {0} & {\frac{\partial w}{\partial z}} \\
\end{pmatrix}
+ 2\mu (\mathbf{e_{ij}})
$$

$$
\implies \tau = \lambda . 
\begin{pmatrix}
{\frac{\partial u}{\partial x}} & {0} & {0} \\
{0} & {\frac{\partial v}{\partial y}} & {0} \\
{0} & {0} & {\frac{\partial w}{\partial z}} \\
\end{pmatrix}
+ 2\mu .\begin{pmatrix}
{2\frac{\partial u}{\partial x}} & 
{\frac{\partial v}{\partial x} + \frac{\partial u}{\partial y}} &
{\frac{\partial w}{\partial x} + \frac{\partial u}{\partial z}} \\
{\frac{\partial u}{\partial y} + \frac{\partial v}{\partial x}} & 
{2\frac{\partial v}{\partial y}} &
{\frac{\partial w}{\partial y} + \frac{\partial v}{\partial z}} \\
{\frac{\partial u}{\partial z} + \frac{\partial w}{\partial x}} &
{\frac{\partial v}{\partial z} + \frac{\partial w}{\partial y}} &
{2\frac{\partial w}{\partial z}} \\
\end{pmatrix}
$$

This lines up with [this](https://wiki.seg.org/wiki/Mathematical_foundation_of_elastic_wave_propagation#equation_L-21)